# 리뷰완료

이 자료는 위키독스 딥 러닝을 이용한 자연어 처리 입문의 GloVe 튜토리얼 자료입니다.  

링크 : https://wikidocs.net/22885  

이 자료는 2021년 10월 14일에 마지막으로 테스트되었습니다.

글로브(Global Vectors for Word Representation, GloVe)는 카운트 기반과 예측 기반을 모두 사용하는 방법론  
2014년에 미국 스탠포드대학에서 개발한 단어 임베딩 방법론입니다.   

앞서 학습하였던 기존의   
카운트 기반의 LSA(Latent Semantic Analysis)와  
예측 기반의 Word2Vec의 단점을 지적하며 이를 보완한다는 목적으로 나왔고,   
실제로도 Word2Vec만큼 뛰어난 성능을 보여줍니다.   

현재까지의 연구에 따르면 단정적으로 Word2Vec와 GloVe 중에서 어떤 것이 더 뛰어나다고 말할 수는 없고,   
이 두 가지 전부를 사용해보고 성능이 더 좋은 것을 사용하는 것이 바람직합니다.

In [1]:
!pip install glove_python_binary

In [2]:
!pip list | grep glove

'grep'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jkm20\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

In [5]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x1a55ea8c250>)

In [6]:
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

In [7]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273424


In [8]:
from glove import Corpus, Glove

corpus = Corpus() 

# 훈련 데이터로부터 GloVe에서 사용할 동시 등장 행렬 생성
corpus.fit(result, window=5)
glove = Glove(no_components=100, learning_rate=0.05)

# 학습에 이용할 쓰레드의 개수는 4로 설정, 에포크는 20.
glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)

Performing 20 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


In [9]:
print(glove.most_similar("man"))

[('woman', 0.9603620760476753), ('guy', 0.8845771865479448), ('girl', 0.8713388800064402), ('kid', 0.8578088462714761)]


In [10]:
print(glove.most_similar("boy"))

[('girl', 0.9385090829685033), ('woman', 0.8500961537984489), ('kid', 0.8309936890172507), ('man', 0.8293959373873518)]


In [11]:
print(glove.most_similar("university"))

[('harvard', 0.8918218549044832), ('mit', 0.8625898726819271), ('stanford', 0.8461804714488959), ('cambridge', 0.8438438652918612)]


In [12]:
print(glove.most_similar("water"))

[('clean', 0.8436546443130409), ('fresh', 0.831762116119486), ('air', 0.8271148031626231), ('electricity', 0.8037353209315635)]


In [13]:
print(glove.most_similar("physics"))

[('chemistry', 0.8976574561251395), ('economics', 0.8802583642416056), ('mathematics', 0.8635247517748392), ('beauty', 0.8560893334600962)]


In [14]:
print(glove.most_similar("muscle"))

[('tissue', 0.8301676961442684), ('nerve', 0.8276105368103941), ('channel', 0.7539728235818708), ('skeletal', 0.7485636064153074)]


In [15]:
print(glove.most_similar("clean"))

[('water', 0.8436546443130409), ('fresh', 0.8366765216081009), ('wind', 0.8003151735341588), ('heat', 0.7886525823618243)]
